In [4]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, AncillaRegister, IBMQ
from qiskit.circuit.library import QFT, GraphState, GroverOperator
from qiskit.qasm import pi
from utils import *


import grover, qaoa, ghz, qft, qft_entangled, shor, w_state, qpe_inexact, qpe_exact, graph_state, vqe

In [5]:
provider = IBMQ.load_account()
backend = provider.backend.ibmq_santiago
c_map = backend.configuration().coupling_map

In [6]:
benchmarks = [grover, qaoa, ghz, qft, qft_entangled, w_state, qpe_inexact, qpe_exact, vqe]

In [7]:
num_qubits_test = 5
n_min = 3
n_max = 7
opt_level = 2

In [ ]:
max_n_arch = max(max(c_map))
for benchmark in benchmarks:
    for n in range(n_min, n_max):
        qc = benchmark.create_circuit(n)
        compiled_without_architecure = get_compiled_circuit(qc, opt_level=opt_level)
        save_as_qasm(compiled_without_architecure, n)
        if (n < max_n_arch):
            compiled_with_architecture = get_compiled_circuit(qc, opt_level=opt_level, c_map=c_map)
            save_as_qasm(compiled_with_architecture, n, True)

Grover Test

In [ ]:
a = grover.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QAOA Test

In [ ]:
a = qaoa.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

GHZ Test

In [ ]:
a = ghz.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Test

In [ ]:
a = qft.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QFT Entangled Test

In [ ]:
a = qft_entangled.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

W-State Test

In [ ]:
a = w_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

QPE Exact and QPE Inexact Test

In [ ]:
a = qpe_exact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

a = qpe_inexact.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)

Graph State Test

In [ ]:
a = graph_state.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)


Shor Test

In [ ]:
a = shor.create_circuit(15)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, None)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, None, True)

QAE Test

In [15]:
import ae

In [16]:
a = ae.create_circuit(num_qubits_test)
b = get_compiled_circuit(a, opt_level=2)
save_as_qasm(b, num_qubits_test)
c = get_compiled_circuit(b, opt_level=2, c_map=c_map)
save_as_qasm(c, num_qubits_test, True)
